# Abstract

This experiment attempts to forecast peak bloom dates by using a temperature-based time-series pattern matching technique. By using historical weather data and bloom observations, the method identifies analogous climate sequences from past years to project future bloom timings.

The framework used is Fast Dynamic Time Warping (FastDTW), which compares recent temperature patterns with historical patterns, determining the most probable position in the bloom cycle. The dataset consists of daily temperature records for many years, formatted to represent the cycles of dormancy and bloom. To improve predictive accuracy, we introduce derived features including cumulative degree days, rolling temperature sums, and sum of squares of temperature values, and more.

When testing the model, partial temperature sequences from historical bloom years are selected and compared in alignment to actual bloom dates. By iteratively sliding time windows over historical records, we can evaluate how closely predicted bloom dates match observed events through minimizng the Euclidean norm. 

Despite the many limitations in this design, it may prove beneficial to track bloom dates in terms of cyles and patterns. This tool of sliding windows to find patterns in the data can allow us to forecast any time-series datasets.

# Introduction

This project aims to predict Cherry Blossom peak bloom dates (PBD) for 2025 at 5 locations around the world. Historical data on Cherry Blossom PBD was used to model bloom cycles, where the beginning of the cycle represents one day after previous PBD, and the final day represents the current PBD. These cycles are compared to one another, mapping similar trends on functions of temperature. In particular, the days leading up to the bloom were closely analyzed, allowing us to theoretically input a slice of current temperature data in order to predict how far along the cycle we are, thus generating an estimate for the future PBD. 


# Setup

This project uses python and related packages. Please ensure python is installed, as well as all other dependencies listed below:

In [ ]:
import pandas as pd 
import numpy as np 
import datetime as dt
import matplotlib.pyplot as plt
from fastdtw import fastdtw
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import Bloom Datasets
washingtondc = pd.read_csv('data/washingtondc.csv')
liestal = pd.read_csv('data/liestal.csv')
kyoto = pd.read_csv('data/kyoto.csv')
vancouver = pd.read_csv('data/vancouver.csv')
newyorkcity = pd.read_csv('data/nyc.csv')

In [ ]:
# Create and engineer relevant features for Washington DC 
# Years not included (missing data): 2000 
wdc_weather_df = pd.read_csv('data/wdc_weather1990_2024.csv')
wdc_weather_df.reset_index(drop=True, inplace=True)
wdc_weather_df['Date'] = pd.to_datetime(wdc_weather_df['Date'])
wdc_weather_df.sort_values(by='Date',inplace=True)
wdc_weather_df.drop(columns=['index'], inplace=True)
wdc_weather_df

In [ ]:
wdc_bloomdates_df = pd.DataFrame(washingtondc)
wdc_bloomdates_df = wdc_bloomdates_df[wdc_bloomdates_df['year'] >= 1990]
wdc_bloomdates_df = wdc_bloomdates_df.drop(wdc_bloomdates_df[wdc_bloomdates_df['year'] == 2000].index)
wdc_bloomdates_df['bloom_date'] = pd.to_datetime(wdc_bloomdates_df['bloom_date'])
wdc_bloomdates_df.reset_index(drop=True, inplace=True)
wdc_bloomdates_df

Now we have two dataframes:

wdc_weather_df, daily weather data 
wdc_bloomdates_df, with bloom dates for each year

Since we want to compare a bloom cycle, i.e from dormancy to bloom, we will count each year's 
bloom as starting from the day after the previous bloom and ending with the current bloom.

temp_of_day_from_bloom_df: 

- rows: day of bloom (1 denotes first day after previous bloom) 
- column: cycle (year)

For example, if the column name is 1992_b, this means that this column the days leading up to the bloom
in 1992. Thus the first row will take place in 1991, after the previous bloom 1991_b. The last value, prior
to any NaN, is the bloom date. 

Each column represents one cycle, from post-bloom dormancy to the next bloom.

We will also take the inverse of the dataframe, and call it temp_of_day_till_bloom_df:

- rows: day till bloom (1 denotes one day prior to the current bloom, and 0 be the bloom itself.)
- column: cycle (year)


In [ ]:
bloom_dates = wdc_bloomdates_df['bloom_date']
temp_of_day_from_bloom_df = pd.DataFrame() 

for i in range(len(bloom_dates) - 1):
    data = wdc_weather_df[(wdc_weather_df['Date'] > bloom_dates[i]) & (wdc_weather_df['Date'] <= bloom_dates[i+1])]
    data.reset_index(drop=True, inplace=True)
    data["day_from_bloom"] = data.index + 1
    pivot_df = data.pivot_table(index='day_from_bloom', values='Temp_avg')
    pivot_df.columns=[str(bloom_dates[i+1].year)+'_b']
    temp_of_day_from_bloom_df = pd.concat([temp_of_day_from_bloom_df, pivot_df], axis=1)

temp_of_day_till_bloom_df = pd.DataFrame()
inv_temp_of_day_from_bloom_df = temp_of_day_from_bloom_df.iloc[::-1]

for col in inv_temp_of_day_from_bloom_df.columns:
    nan_count = inv_temp_of_day_from_bloom_df[col].isna().sum()

    # Remove NaN
    temp_of_day_till_bloom_df[col] = inv_temp_of_day_from_bloom_df[col].iloc[nan_count:].reset_index(drop=True)

temp_of_day_till_bloom_df = temp_of_day_till_bloom_df.head(330)
temp_of_day_till_bloom_df['days_till_bloom'] = temp_of_day_till_bloom_df.index
temp_of_day_till_bloom_df

Now, let's add some more complicated features onto the dataframe. We will look at functions of temperature, as it is theorized that bloom cycles occur after certain thresholds of temperature are met. The following functions were used as features:

All data is created and stored in the new dataframe, df.

Each feature name begins with the year of bloom, followed by the following suffix. For example, the 1991 bloom cycle will have the features:

- 1991_b : daily average Temperature
- 1991_b_sum_sq : sum of squared daily temperature, rolling over last 21 days 
- 1991_b_sum_log : sum of log daily temperature, rolling over last 21 days

... and so on. Below are definitions of all features

| Feature     | 10      |
|------|-------------|
| _sum_sq    | sum of squared daily temperature, rolling over last 21 days     |
| _sum_log   | sum of log daily temperature, rolling over last 21 days     |
| _sum_dd32  | sum of number of days where temperature exceeds 32 F, rolling over last 21 days     |
| _sum_dd50  | sum of number of days where temperature exceeds 50 F, rolling over last 21 days     | 
| _sum_last14    | sum of daily temperature, rolling over last 14 days     |
| _norm_dd_32   | sum of days where days above 32F are (+1) and days below are (-1), rolling over last 21 days     |
| _norm_dd_50  |  sum of days where days above 50F are (+1) and days below are (-1), rolling over last 21 days     |
| _rolling_freeze  | sum of number of days where temperature is below 32 F, rolling over last 21 days      | 
| _rolling_chill  | sum of number of days where temperature is between 32 F and 50 F, rolling over last 21 days     | 
| _rolling_temp_change  | temperature change between last 7 days     | 


In [ ]:
df = temp_of_day_till_bloom_df.copy()

# Functions of temperature for each bloom cycle
for col in df.columns:
    if col.endswith('_b'):
        df[f'{col}_sum_sq'] = df[col][::-1].pow(2).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_sum_log'] = df[col][::-1].apply(lambda x: np.log(x + 1)).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_sum_dd32'] = (df[col][::-1] > 32).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_sum_dd50'] = (df[col][::-1] > 50).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_sum_last14'] = df[col][::-1].rolling(window=14, min_periods=1).sum()[::-1]
        df[f'{col}_norm_dd_32'] = ((df[col] > 32).astype(int) * 1 + (df[col] <= 32).astype(int) * -1).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_norm_dd_50'] = ((df[col] > 50).astype(int) * 1 + (df[col] <= 50).astype(int) * -1).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_rolling_freeze'] = (df[col][::-1] < 32).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_rolling_chill'] = ((df[col][::-1] >= 32) & (df[col][::-1] < 50)).rolling(window=21, min_periods=1).sum()[::-1]
        df[f'{col}_rolling_temp_change'] = df[col][::-1].diff(periods=7)[::-1]

cols = ['days_till_bloom'] + [c for c in df.columns if c != 'days_till_bloom']
df = df[cols]
df = df.head(80)
df

Let's see what our data looks like.


In [ ]:
# Show raw data

plt.figure(figsize=(10, 10))

for year in temp_of_day_from_bloom_df.columns:
    bloom_cycle = df[year]
    plt.plot(bloom_cycle.index, bloom_cycle.values, label=str(year))

plt.xlabel('Number of Days Till Bloom')
plt.ylabel('Temperature in F')
plt.title('Temperature Over Bloom Cycles')
plt.legend(title="Year", bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
window = 7

plt.figure(figsize=(10, 10))  

for year in temp_of_day_from_bloom_df.columns:
    bloom_cycle = df[year]
    smoothed = bloom_cycle.rolling(window=window).mean()  
    plt.plot(smoothed.index, smoothed.values, label=str(year))

plt.xlabel('Number of Days Till Bloom')
plt.ylabel('Temperature (Temp Avg), in F')
plt.title('Smoothed Temperature Change Over Bloom Cycles')
plt.legend(title="Year", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

Looking at continuous functions of temperature:

In [ ]:
feature_suffixes_cont = ['_sum_sq', '_sum_log', '_sum_last14', '_rolling_temp_change']
for suffix in feature_suffixes_cont:
    cv = df[[col for col in df.columns if col.endswith(suffix)]]
    plt.figure(figsize=(10, 3)) 
    plt.xlabel('Number of Days Till Bloom')
    plt.xticks(np.arange(0, 115, 5))
    plt.title('Patterns of ' + suffix)
    plt.ylabel(suffix)
    for year in cv.columns:
        bloom_cycle = df[year]
        plt.plot(bloom_cycle.index,bloom_cycle.values)
    plt.show()

Now discrete functions of temperature:

In [ ]:
feature_suffixes_disc = ['_sum_dd32', '_sum_dd50','_norm_dd_32', '_norm_dd_50','_rolling_freeze', '_rolling_chill',]
for suffix in feature_suffixes_disc:
    cv = df[[col for col in df.columns if col.endswith(suffix)]]
    plt.figure(figsize=(10, 3))
    plt.xlabel('Number of Days Till Bloom')
    plt.xticks(np.arange(0, 115, 5))
    plt.title('Patterns of ' + suffix)
    plt.ylabel(suffix)
    for year in cv.columns:
        bloom_cycle = df[year]
        plt.plot(bloom_cycle.index,bloom_cycle.values,
                 marker='o', linestyle='-', markersize=4, label=year
)
    plt.show()

# Prediction

Now let's define some methods that will help us experiment and predict on data. 

In [ ]:
# DEFINE METHODS FOR PREDICTING BLOOM FOR 2025

def extract_historical_feature_sets(df, feature_suffixes):
    """
    Returns a dict where keys are feature suffixes, and values are DF of those suffixes.
    """
    feature_sets = {}
    for suffix in feature_suffixes:
        historical_features = df[[col for col in df.columns if col.endswith(suffix)]]
        feature_sets[suffix] = historical_features
    return feature_sets

def predict_from_arrays(historical_feature_sets, test_sequences):
    """
    - historical_feature_sets: dict where keys are feature suffixes, and values are DF of those suffixes
    - test_sequences: dict where keys are feature suffixes, values are test numpy arrays.
        
    Returns:
        - results: dict with best-matching year, best days_till_bloom index, and distance for each feature type.
    """
    results = {}
    
    for suffix, test_seq in test_sequences.items():
        best_match = None
        best_distance = float('inf')
        best_days_till_bloom = None

        historical_data = historical_feature_sets[suffix]

        for year in historical_data.columns:
            hist_seq = historical_data[year].values.flatten()
                        
            # FastDTW over a sliding window
            for days_till_bloom in range(len(hist_seq) - len(test_seq) + 1):
                window = hist_seq[days_till_bloom: days_till_bloom + len(test_seq)]
                distance, _ = fastdtw(window, test_seq, dist=2)
                
                if distance < best_distance:
                    best_distance = distance
                    best_match = year
                    best_days_till_bloom = days_till_bloom
        
        results[suffix] = (best_match, best_days_till_bloom, best_distance)
        #print(f"Feature: {suffix}, Best Match: {best_match}, Days_till_bloom: {best_days_till_bloom}, Distance: {best_distance}")

    return results

# feature suffixes
feature_suffixes = ['_sum_sq', '_sum_log', '_sum_dd32', '_sum_dd50', '_sum_last14', 
    '_norm_dd_32', '_norm_dd_50','_rolling_freeze', '_rolling_chill', '_rolling_temp_change'
]

# historical feature sets
historical_feature_sets = extract_historical_feature_sets(df, feature_suffixes)

Here, the prediction function uses FastDTW to compare vectors of time series. It finds the lowest Euclidean norm (L2) between a given input and a historical data. This slice of temperature input is thought to be "sliding" across the time, checking for patterns and attempting to match it to a historical bloom cycle. After completing the computations, it returns the lowest L2 norm and what the implied days_till_bloom is. This can be used to predict where we currently are in the cycle, which intuitively allows us to estimate when it will bloom. This process was used due to the fact that future temperature data would not need to be modeled or predicted.


In [ ]:
# BUILDING 2025 TEMPERATURE SET DATAFRAME FOR 5 LOCATIONS

wdc_weather2025 = pd.read_csv('data/wdc_weather2025.csv')
liestal_weather2025 = pd.read_csv('data/liestal_weather2025.csv')
kyoto_weather2025 = pd.read_csv('data/kyoto_weather2025.csv')
vancouver_weather2025 = pd.read_csv('data/vancouver_weather2025.csv')
nyc_weather2025 = pd.read_csv('data/nyc_weather2025.csv')

wdc_weather2025['Date'] = pd.to_datetime(wdc_weather2025['Date'])
liestal_weather2025['Date'] = pd.to_datetime(liestal_weather2025['Date'])
kyoto_weather2025['Date'] = pd.to_datetime(kyoto_weather2025['Date'])
vancouver_weather2025['Date'] = pd.to_datetime(vancouver_weather2025['Date'])
nyc_weather2025['Date'] = pd.to_datetime(nyc_weather2025['Date'])

wdc_weather2025.name = "Washington DC"
liestal_weather2025.name = "Liestal"
kyoto_weather2025.name = "Kyoto"
vancouver_weather2025.name = "Vancouver"
nyc_weather2025.name = "NYC"

all_locations = [wdc_weather2025,liestal_weather2025, kyoto_weather2025, vancouver_weather2025, nyc_weather2025]


This function computes the temperature prediction for all 5 locations:

In [ ]:
data = []
for loc in all_locations:
    temp_2025_dc = loc.sort_values(by='Date', ascending=False)[['Temp_avg']]
    temp_2025_dc.columns=['2025_b']
    for col in temp_2025_dc.columns:
        if col.endswith('_b'):
            temp_2025_dc[f'{col}_sum_sq'] = temp_2025_dc[col][::-1].pow(2).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_sum_log'] = temp_2025_dc[col][::-1].apply(lambda x: np.log(x + 1)).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_sum_dd32'] = (temp_2025_dc[col][::-1] > 32).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_sum_dd50'] = (temp_2025_dc[col][::-1] > 50).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_sum_last14'] = temp_2025_dc[col][::-1].rolling(window=14, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_norm_dd_32'] = ((temp_2025_dc[col] > 32).astype(int) * 1 + (temp_2025_dc[col] <= 32).astype(int) * -1).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_norm_dd_50'] = ((temp_2025_dc[col] > 50).astype(int) * 1 + (temp_2025_dc[col] <= 50).astype(int) * -1).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_rolling_freeze'] = (temp_2025_dc[col][::-1] < 32).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_rolling_chill'] = ((temp_2025_dc[col][::-1] >= 32) & (temp_2025_dc[col][::-1] < 50)).rolling(window=21, min_periods=1).sum()[::-1]
            temp_2025_dc[f'{col}_rolling_temp_change'] = temp_2025_dc[col][::-1].diff(periods=7)[::-1]
    temp_2025_dc = temp_2025_dc.head(50)

    all_results = []
    intervals = [10,20,30,40]

    for interval in intervals:
        temp_2025_dc_df = temp_2025_dc.head(interval)

        test_sequences = {
            '_sum_sq': np.array(temp_2025_dc_df['2025_b_sum_sq']),
            '_sum_log': np.array(temp_2025_dc_df['2025_b_sum_log']),
            '_sum_dd32': np.array(temp_2025_dc_df['2025_b_sum_dd32']),
            '_sum_dd50': np.array(temp_2025_dc_df['2025_b_sum_dd50']),
            '_sum_last14': np.array(temp_2025_dc_df['2025_b_sum_last14']),
            '_norm_dd_32': np.array(temp_2025_dc_df['2025_b_norm_dd_32']),
            '_norm_dd_50': np.array(temp_2025_dc_df['2025_b_norm_dd_50']),
            '_rolling_freeze': np.array(temp_2025_dc_df['2025_b_rolling_freeze']),
            '_rolling_chill': np.array(temp_2025_dc_df['2025_b_rolling_chill']),
            '_rolling_temp_change': np.array(temp_2025_dc_df['2025_b_rolling_temp_change']),

        }

        all_results.append((interval, predict_from_arrays(historical_feature_sets, test_sequences)))

    #all_results
    print("Results for " + loc.name)

    for result in all_results:
        print(result)

    arrays = []
    for j in range(len(feature_suffixes)):
        arrays.append([])

    for i in range(4):
        for j in range(len(feature_suffixes)):
            arrays[j].append(all_results[i][1][feature_suffixes[j]][1])
    
    data.append((loc.name, arrays))
    
    print("----- ----- -----")


After computing the estimated days_till_bloom for differing interval lengths, we can compare the estimated days. There are many outliers in the prediction, and the predictions can be wildly different at times. The chosen dates were picked in a non-rigorous, intuitive approach. While this may not be an ideal method for repeatability, for this specific prediction it might be okay. That's because many of the high variance points are very unlikely; for example, points like 0 or 50 mean that the model thinks the trees are blooming today or in 50 days. We can reason intuitively that neither of these cases are/will happen, based on historical blooms. Thus, the median values we are left with are used to create the prediction and prediction intervals.


Table for Washington DC

- columns represent the size of the testing interval.
- Values represent the predicted days_till_bloom 


| Feature     | 10      | 20    | 30    |  40   |
|------------|--------:|--------:|--------:|--------:|
| _sum_sq    | 24     | 37     | 34     | 32     |
| _sum_log   | 41     | 33     | 22     | 21     |
| _sum_dd32  |  0     |  6     |  1     | 40     |
| _sum_dd50  | 31     | 17     | 36     | 18     |

...and more values. \n


In the end, the final predictions for bloom dates (day of year) are:


| location      | prediction      | lower       | upper     |
|--------------|----------------:|------------:|------------:|
| washingtondc |             88  |         84  |         91  |
| liestal      |             85  |         83  |         90  |
| kyoto        |             91  |         88  |         97  |
| vancouver    |             88  |         84  |         94  |
| newyorkcity  |             95  |         93  |         97  |



# Limitations

While the initial results seem like they have potential, it is important to mention the many limitations of this modeling.

- Data set only trained on Washington DC temperatures; may not be comparable to other countries
- Training on more years past 1990
- Differing cycle lengths may impact prediction
- Low historical training data for places like NYC 
- Different breeds of cherry blossoms might have different cycles and cycle patterns
- Not accounting for special impactful events (storms, dry spells, etc)
- Likely better results using other variables alongside temperature (emissions, sunlight, rainfall)
- Predictions and predictions intervals not chosen using empirical and repeatable method 
